In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from collections import Counter
from imblearn.under_sampling import TomekLinks
tl= TomekLinks()

X_tl, y_tl= tl.fit_resample(X, y)
print(f'Tomek Links: {Counter(y_tl)}')

Tomek Links: Counter({0: 41318, 1: 4561})


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

X_tl_train, y_tl_train= tl.fit_resample(X_train, y_train)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_tl= LogisticRegression(random_state=15)
lr_tl.fit(X_tl_train, y_tl_train)
y_pred_lr_tl= lr_tl.predict(X_test)
y_pred_prob_lr_tl= lr_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_tl, labels= lr_tl.classes_)
print(classification_report(y_test, y_pred_lr_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_tl))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.54      0.17      0.26       912

    accuracy                           0.91      9429
   macro avg       0.73      0.58      0.60      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5397923875432526
Accuracy: 0.9057164068299925
AUC: 0.8542947644442798


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_tl= DecisionTreeClassifier(random_state=15)
dt_tl.fit(X_tl_train, y_tl_train)
y_pred_dt_tl= dt_tl.predict(X_test)
y_pred_prob_dt_tl= dt_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_tl, labels= dt_tl.classes_)
print(classification_report(y_test, y_pred_dt_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_tl))

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      8517
           1       0.26      0.32      0.29       912

    accuracy                           0.85      9429
   macro avg       0.59      0.61      0.60      9429
weighted avg       0.86      0.85      0.85      9429

Precision: 0.2579505300353357
Accuracy: 0.845158553399088
AUC: 0.6101113691090471


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_tl= GridSearchCV(estimator= dt_tl, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_tl.fit(X_tl_train, y_tl_train)

print("Best Parameters:", gs_dt_tl.best_params_)
print("Best Precision Score:", gs_dt_tl.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.5267495328903364


In [12]:
tuned_dt_tl= gs_dt_tl.best_estimator_
y_pred_tuned_dt_tl= tuned_dt_tl.predict(X_test)
y_pred_prob_tuned_dt_tl= tuned_dt_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_tl, labels= tuned_dt_tl.classes_)
print(classification_report(y_test, y_pred_tuned_dt_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_tl))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.34      0.05      0.08       912

    accuracy                           0.90      9429
   macro avg       0.63      0.52      0.51      9429
weighted avg       0.85      0.90      0.86      9429

Precision: 0.344
Accuracy: 0.8991409481387209
AUC: 0.7839179741651887


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_tl= KNeighborsClassifier()
knn_tl.fit(X_tl_train, y_tl_train)
y_pred_knn_tl= knn_tl.predict(X_test)
y_pred_prob_knn_tl= knn_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_tl, labels= knn_tl.classes_)
print(classification_report(y_test, y_pred_knn_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_tl))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      8517
           1       0.35      0.14      0.21       912

    accuracy                           0.89      9429
   macro avg       0.63      0.56      0.57      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.3548387096774194
Accuracy: 0.8918230989500477
AUC: 0.7117707953545953


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_tl= GridSearchCV(estimator= knn_tl, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_tl.fit(X_tl_train, y_tl_train)
print("Best Parameters:", gs_knn_tl.best_params_)
print("Best Precision Score:", gs_knn_tl.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 38, 'weights': 'uniform'}
Best Precision Score: 0.7343223443223443


In [15]:
tuned_knn_tl= gs_knn_tl.best_estimator_
y_pred_tuned_knn_tl= tuned_knn_tl.predict(X_test)
y_pred_prob_tuned_knn_tl= tuned_knn_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_tl, labels= tuned_knn_tl.classes_)
print(classification_report(y_test, y_pred_tuned_knn_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_tl))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.47      0.02      0.04       912

    accuracy                           0.90      9429
   macro avg       0.69      0.51      0.49      9429
weighted avg       0.86      0.90      0.86      9429

Precision: 0.47368421052631576
Accuracy: 0.9030650121964153
AUC: 0.8172445743188546


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_tl= GaussianNB()
nb_tl.fit(X_tl_train, y_tl_train)
y_pred_nb_tl= nb_tl.predict(X_test)
y_pred_prob_nb_tl= nb_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_tl, labels= nb_tl.classes_)
print(classification_report(y_test, y_pred_nb_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_tl))

              precision    recall  f1-score   support

           0       0.97      0.68      0.80      8517
           1       0.21      0.79      0.33       912

    accuracy                           0.69      9429
   macro avg       0.59      0.74      0.57      9429
weighted avg       0.90      0.69      0.75      9429

Precision: 0.20999128666860295
Accuracy: 0.6914837204369498
AUC: 0.8060199904628309


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_tl= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc_tl.fit(X_tl_train, y_tl_train)
y_pred_svc_tl= svc_tl.predict(X_test)
y_pred_prob_svc_tl= svc_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_tl, labels= svc_tl.classes_)
print(classification_report(y_test, y_pred_svc_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_tl))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.16      0.01      0.01       912

    accuracy                           0.90      9429
   macro avg       0.53      0.50      0.48      9429
weighted avg       0.83      0.90      0.86      9429

Precision: 0.15625
Accuracy: 0.9009438964895535
AUC: 0.7044818386961886


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_tl= RandomForestClassifier(random_state=15)
rf_tl.fit(X_tl_train, y_tl_train)
y_pred_rf_tl= rf_tl.predict(X_test)
y_pred_prob_rf_tl= rf_tl.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_tl, labels= rf_tl.classes_)
print(classification_report(y_test, y_pred_rf_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_tl))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.43      0.11      0.18       912

    accuracy                           0.90      9429
   macro avg       0.67      0.55      0.56      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.43478260869565216
Accuracy: 0.9000954502068088
AUC: 0.8243878599869405


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_tl= GridSearchCV(estimator= rf_tl, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_tl.fit(X_tl_train, y_tl_train)
print("Best Parameters:", gs_tuned_rf_tl.best_params_)
print("Best Precision Score:", gs_tuned_rf_tl.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.6952780590207205


In [21]:
tuned_rf_tl= gs_tuned_rf_tl.best_estimator_
y_pred_tuned_rf_tl= tuned_rf_tl.predict(X_test)
y_pred_prob_tuned_rf_tl= tuned_rf_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_tl, labels= tuned_rf_tl.classes_)
print(classification_report(y_test, y_pred_tuned_rf_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_tl))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      8517
           1       0.58      0.04      0.08       912

    accuracy                           0.90      9429
   macro avg       0.74      0.52      0.52      9429
weighted avg       0.88      0.90      0.87      9429

Precision: 0.5797101449275363
Accuracy: 0.9044437374058755
AUC: 0.8495216738864891


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_tl= AdaBoostClassifier(random_state=15)
ada_tl.fit(X_tl_train, y_tl_train)
y_pred_ada_tl= ada_tl.predict(X_test)
y_pred_prob_ada_tl= ada_tl.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_tl, labels= ada_tl.classes_)
print(classification_report(y_test, y_pred_ada_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_tl))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.53      0.19      0.28       912

    accuracy                           0.91      9429
   macro avg       0.73      0.59      0.61      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5325077399380805
Accuracy: 0.9055042952593064
AUC: 0.8533640278781962


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_tl= GridSearchCV(estimator= ada_tl, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_tl.fit(X_tl_train, y_tl_train)
print("Best Parameters:", gs_ada_tl.best_params_)
print("Best Precision Score:", gs_ada_tl.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 100}
Best Precision Score: 0.5595458423696151


In [25]:
tuned_ada_tl= gs_ada_tl.best_estimator_
y_pred_tuned_ada_tl= tuned_ada_tl.predict(X_test)
y_pred_prob_tuned_ada_tl= tuned_ada_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_tl, labels= tuned_ada_tl.classes_)
print(classification_report(y_test, y_pred_tuned_ada_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_tl))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.54      0.18      0.27       912

    accuracy                           0.91      9429
   macro avg       0.73      0.58      0.61      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5419354838709678
Accuracy: 0.9060345741860218
AUC: 0.8519498670357942


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_tl= GradientBoostingClassifier(random_state=15)
grb_tl.fit(X_tl_train, y_tl_train)
y_pred_grb_tl= grb_tl.predict(X_test)
y_pred_prob_grb_tl= grb_tl.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_tl, labels= grb_tl.classes_)
print(classification_report(y_test, y_pred_grb_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_tl))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      8517
           1       0.52      0.15      0.23       912

    accuracy                           0.90      9429
   macro avg       0.72      0.57      0.59      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.51953125
Accuracy: 0.9043376816205324
AUC: 0.8537664093896828


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_tl= GridSearchCV(estimator= grb_tl, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_tl.fit(X_tl_train, y_tl_train)

print("Best Parameters:", gs_grb_tl.best_params_)
print("Best Precision Score:", gs_grb_tl.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.5793441233357546


In [28]:
tuned_grb_tl= gs_grb_tl.best_estimator_
y_pred_tuned_grb_tl= tuned_grb_tl.predict(X_test)
y_pred_prob_tuned_grb_tl= tuned_grb_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_tl, labels= tuned_grb_tl.classes_)
print(classification_report(y_test, y_pred_tuned_grb_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_tl))

              precision    recall  f1-score   support

           0       0.92      0.99      0.95      8517
           1       0.52      0.15      0.23       912

    accuracy                           0.90      9429
   macro avg       0.72      0.57      0.59      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.51953125
Accuracy: 0.9043376816205324
AUC: 0.8537664093896828


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_tl= XGBClassifier(random_state=15)
xgb_tl.fit(X_tl_train, y_tl_train)
y_pred_xgb_tl= xgb_tl.predict(X_test)
y_pred_prob_xgb_tl= xgb_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_tl, labels= xgb_tl.classes_)
print(classification_report(y_test, y_pred_xgb_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_tl))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.48      0.18      0.26       912

    accuracy                           0.90      9429
   macro avg       0.70      0.58      0.60      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.4809384164222874
Accuracy: 0.9018983985576413
AUC: 0.8318473347422801


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_tl= GridSearchCV(estimator= xgb_tl, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_tl.fit(X_tl_train, y_tl_train)

print("Best Parameters:", gs_xgb_tl.best_params_)
print("Best Precision Score:", gs_xgb_tl.best_score_)

tuned_xgb_tl= gs_xgb_tl.best_estimator_
y_pred_tuned_xgb_tl= tuned_xgb_tl.predict(X_test)
y_pred_prob_tuned_xgb_tl= tuned_xgb_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_tl, labels= tuned_xgb_tl.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_tl))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.01}
Best Precision Score: 0.7544444444444445
              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.53      0.01      0.02       912

    accuracy                           0.90      9429
   macro avg       0.72      0.50      0.48      9429
weighted avg       0.87      0.90      0.86      9429

Precision: 0.5294117647058824
Accuracy: 0.9033831795524446
AUC: 0.8395735618546191


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_tl= LGBMClassifier(random_state=15)
lgm_tl.fit(X_tl_train, y_tl_train)
y_pred_lgm_tl= lgm_tl.predict(X_test)
y_pred_prob_lgm_tl= lgm_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_tl, labels= lgm_tl.classes_)
print(classification_report(y_test, y_pred_lgm_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_tl))

[LightGBM] [Info] Number of positive: 3649, number of negative: 32789
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004479 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 251
[LightGBM] [Info] Number of data points in the train set: 36438, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100143 -> initscore=-2.195640
[LightGBM] [Info] Start training from score -2.195640
              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.52      0.14      0.22       912

    accuracy                           0.90      9429
   macro avg       0.72      0.56      0.58      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.5186721991701245
Accuracy: 0.9042316258351893
AUC: 0.8503889409004488


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_tl= GridSearchCV(estimator= lgm_tl, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_tl.fit(X_tl_train, y_tl_train)

print("Best Parameters:", gs_lgm_tl.best_params_)
print("Best Precision Score:", gs_lgm_tl.best_score_)

tuned_lgm_tl= gs_lgm_tl.best_estimator_
y_pred_tuned_lgm_tl= tuned_lgm_tl.predict(X_test)
y_pred_prob_tuned_lgm_tl= tuned_lgm_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_tl, labels= tuned_lgm_tl.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_tl))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 3649, number of negative: 32789
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008744 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 251
[LightGBM] [Info] Number of data points in the train set: 36438, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100143 -> initscore=-2.195640
[LightGBM] [Info] Start training from score -2.195640
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 50, 'num_leaves': 127, 'reg_alpha': 0.5}
Best Precision Score: 0.9
[LightGBM]

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_tl= CatBoostClassifier(random_state=15)
cat_tl.fit(X_tl_train, y_tl_train)
y_pred_cat_tl= cat_tl.predict(X_test)
y_pred_prob_cat_tl= cat_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_tl, labels= cat_tl.classes_)
print(classification_report(y_test, y_pred_cat_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_tl))

Learning rate set to 0.047832
0:	learn: 0.6401791	total: 155ms	remaining: 2m 34s
1:	learn: 0.5954784	total: 171ms	remaining: 1m 25s
2:	learn: 0.5593443	total: 184ms	remaining: 1m 1s
3:	learn: 0.5209370	total: 199ms	remaining: 49.5s
4:	learn: 0.4914234	total: 213ms	remaining: 42.4s
5:	learn: 0.4636948	total: 226ms	remaining: 37.4s
6:	learn: 0.4407564	total: 236ms	remaining: 33.5s
7:	learn: 0.4215793	total: 249ms	remaining: 30.8s
8:	learn: 0.4028684	total: 259ms	remaining: 28.6s
9:	learn: 0.3861011	total: 268ms	remaining: 26.5s
10:	learn: 0.3726944	total: 277ms	remaining: 24.9s
11:	learn: 0.3603869	total: 286ms	remaining: 23.5s
12:	learn: 0.3505967	total: 293ms	remaining: 22.3s
13:	learn: 0.3413830	total: 301ms	remaining: 21.2s
14:	learn: 0.3331191	total: 308ms	remaining: 20.2s
15:	learn: 0.3253116	total: 316ms	remaining: 19.4s
16:	learn: 0.3181186	total: 324ms	remaining: 18.7s
17:	learn: 0.3127963	total: 331ms	remaining: 18.1s
18:	learn: 0.3074032	total: 339ms	remaining: 17.5s
19:	learn

169:	learn: 0.2347178	total: 2.58s	remaining: 12.6s
170:	learn: 0.2346403	total: 2.59s	remaining: 12.6s
171:	learn: 0.2345402	total: 2.61s	remaining: 12.6s
172:	learn: 0.2344685	total: 2.62s	remaining: 12.5s
173:	learn: 0.2343884	total: 2.63s	remaining: 12.5s
174:	learn: 0.2343320	total: 2.65s	remaining: 12.5s
175:	learn: 0.2342632	total: 2.67s	remaining: 12.5s
176:	learn: 0.2341740	total: 2.68s	remaining: 12.5s
177:	learn: 0.2341022	total: 2.69s	remaining: 12.4s
178:	learn: 0.2340159	total: 2.71s	remaining: 12.4s
179:	learn: 0.2339675	total: 2.72s	remaining: 12.4s
180:	learn: 0.2338827	total: 2.73s	remaining: 12.4s
181:	learn: 0.2337891	total: 2.75s	remaining: 12.4s
182:	learn: 0.2336809	total: 2.76s	remaining: 12.3s
183:	learn: 0.2336134	total: 2.77s	remaining: 12.3s
184:	learn: 0.2335354	total: 2.79s	remaining: 12.3s
185:	learn: 0.2334669	total: 2.8s	remaining: 12.3s
186:	learn: 0.2334197	total: 2.81s	remaining: 12.2s
187:	learn: 0.2333463	total: 2.83s	remaining: 12.2s
188:	learn: 0

340:	learn: 0.2215097	total: 4.55s	remaining: 8.79s
341:	learn: 0.2214370	total: 4.56s	remaining: 8.78s
342:	learn: 0.2213823	total: 4.57s	remaining: 8.76s
343:	learn: 0.2213208	total: 4.58s	remaining: 8.73s
344:	learn: 0.2212533	total: 4.59s	remaining: 8.71s
345:	learn: 0.2211845	total: 4.6s	remaining: 8.69s
346:	learn: 0.2211313	total: 4.61s	remaining: 8.67s
347:	learn: 0.2210443	total: 4.62s	remaining: 8.65s
348:	learn: 0.2209723	total: 4.62s	remaining: 8.63s
349:	learn: 0.2208792	total: 4.63s	remaining: 8.6s
350:	learn: 0.2207853	total: 4.64s	remaining: 8.58s
351:	learn: 0.2207422	total: 4.65s	remaining: 8.56s
352:	learn: 0.2206731	total: 4.66s	remaining: 8.54s
353:	learn: 0.2206048	total: 4.67s	remaining: 8.52s
354:	learn: 0.2205343	total: 4.68s	remaining: 8.5s
355:	learn: 0.2204405	total: 4.69s	remaining: 8.48s
356:	learn: 0.2203928	total: 4.69s	remaining: 8.46s
357:	learn: 0.2203306	total: 4.7s	remaining: 8.44s
358:	learn: 0.2202429	total: 4.71s	remaining: 8.42s
359:	learn: 0.22

512:	learn: 0.2111268	total: 6.15s	remaining: 5.83s
513:	learn: 0.2110571	total: 6.16s	remaining: 5.82s
514:	learn: 0.2110449	total: 6.17s	remaining: 5.81s
515:	learn: 0.2110186	total: 6.17s	remaining: 5.79s
516:	learn: 0.2109665	total: 6.18s	remaining: 5.78s
517:	learn: 0.2109327	total: 6.19s	remaining: 5.76s
518:	learn: 0.2108758	total: 6.2s	remaining: 5.75s
519:	learn: 0.2108210	total: 6.21s	remaining: 5.73s
520:	learn: 0.2107654	total: 6.23s	remaining: 5.73s
521:	learn: 0.2106821	total: 6.24s	remaining: 5.71s
522:	learn: 0.2106062	total: 6.25s	remaining: 5.7s
523:	learn: 0.2105319	total: 6.26s	remaining: 5.69s
524:	learn: 0.2104616	total: 6.27s	remaining: 5.68s
525:	learn: 0.2103897	total: 6.28s	remaining: 5.66s
526:	learn: 0.2103146	total: 6.29s	remaining: 5.65s
527:	learn: 0.2102550	total: 6.3s	remaining: 5.63s
528:	learn: 0.2101987	total: 6.31s	remaining: 5.62s
529:	learn: 0.2101881	total: 6.32s	remaining: 5.61s
530:	learn: 0.2101632	total: 6.33s	remaining: 5.59s
531:	learn: 0.2

678:	learn: 0.2027082	total: 7.7s	remaining: 3.64s
679:	learn: 0.2026351	total: 7.71s	remaining: 3.63s
680:	learn: 0.2026072	total: 7.72s	remaining: 3.62s
681:	learn: 0.2025419	total: 7.73s	remaining: 3.6s
682:	learn: 0.2024962	total: 7.74s	remaining: 3.59s
683:	learn: 0.2024344	total: 7.75s	remaining: 3.58s
684:	learn: 0.2023815	total: 7.76s	remaining: 3.57s
685:	learn: 0.2023293	total: 7.76s	remaining: 3.55s
686:	learn: 0.2022698	total: 7.77s	remaining: 3.54s
687:	learn: 0.2022074	total: 7.78s	remaining: 3.53s
688:	learn: 0.2021521	total: 7.79s	remaining: 3.52s
689:	learn: 0.2020756	total: 7.8s	remaining: 3.51s
690:	learn: 0.2020058	total: 7.81s	remaining: 3.49s
691:	learn: 0.2019536	total: 7.82s	remaining: 3.48s
692:	learn: 0.2019168	total: 7.83s	remaining: 3.47s
693:	learn: 0.2018722	total: 7.84s	remaining: 3.46s
694:	learn: 0.2018083	total: 7.85s	remaining: 3.44s
695:	learn: 0.2017481	total: 7.86s	remaining: 3.43s
696:	learn: 0.2016855	total: 7.87s	remaining: 3.42s
697:	learn: 0.2

846:	learn: 0.1949089	total: 9.27s	remaining: 1.67s
847:	learn: 0.1948834	total: 9.28s	remaining: 1.66s
848:	learn: 0.1948398	total: 9.29s	remaining: 1.65s
849:	learn: 0.1947932	total: 9.3s	remaining: 1.64s
850:	learn: 0.1947194	total: 9.31s	remaining: 1.63s
851:	learn: 0.1946655	total: 9.31s	remaining: 1.62s
852:	learn: 0.1945894	total: 9.32s	remaining: 1.61s
853:	learn: 0.1945280	total: 9.33s	remaining: 1.59s
854:	learn: 0.1944720	total: 9.34s	remaining: 1.58s
855:	learn: 0.1944175	total: 9.35s	remaining: 1.57s
856:	learn: 0.1943916	total: 9.36s	remaining: 1.56s
857:	learn: 0.1943513	total: 9.37s	remaining: 1.55s
858:	learn: 0.1942973	total: 9.37s	remaining: 1.54s
859:	learn: 0.1942648	total: 9.38s	remaining: 1.53s
860:	learn: 0.1942486	total: 9.39s	remaining: 1.52s
861:	learn: 0.1942020	total: 9.4s	remaining: 1.5s
862:	learn: 0.1941504	total: 9.41s	remaining: 1.49s
863:	learn: 0.1940968	total: 9.42s	remaining: 1.48s
864:	learn: 0.1940380	total: 9.43s	remaining: 1.47s
865:	learn: 0.1

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_tl= GridSearchCV(estimator= cat_tl, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_tl.fit(X_tl_train, y_tl_train)

print("Best Parameters:", gs_cat_tl.best_params_)
print("Best Precision Score:", gs_cat_tl.best_score_)

tuned_cat_tl= gs_cat_tl.best_estimator_
y_pred_tuned_cat_tl= tuned_cat_tl.predict(X_test)
y_pred_prob_tuned_cat_tl= tuned_cat_tl.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_tl, labels= tuned_cat_tl.classes_)
print(classification_report(y_test, y_pred_tuned_cat_tl, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_tl, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_tl))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_tl))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6821972	total: 8.41ms	remaining: 833ms
1:	learn: 0.6722783	total: 17.5ms	remaining: 856ms
2:	learn: 0.6619658	total: 24.1ms	remaining: 780ms
3:	learn: 0.6532827	total: 29.7ms	remaining: 713ms
4:	learn: 0.6439909	total: 35.2ms	remaining: 669ms
5:	learn: 0.6353072	total: 41.1ms	remaining: 645ms
6:	learn: 0.6265915	total: 46.5ms	remaining: 618ms
7:	learn: 0.6184540	total: 52.8ms	remaining: 607ms
8:	learn: 0.6099927	total: 59.5ms	remaining: 601ms
9:	learn: 0.6021237	total: 64.7ms	remaining: 582ms
10:	learn: 0.5945818	total: 70.4ms	remaining: 569ms
11:	learn: 0.5863186	total: 76.3ms	remaining: 560ms
12:	learn: 0.5783712	total: 81.7ms	remaining: 546ms
13:	learn: 0.5707426	total: 87.3ms	remaining: 536ms
14:	learn: 0.5636746	total: 93.5ms	remaining: 530ms
15:	learn: 0.5561447	total: 99.6ms	remaining: 523ms
16:	learn: 0.5495254	total: 105ms	remaining: 514ms
17:	learn: 0.5423358	total: 111ms	remaining: 505ms
18:	learn: 0.5

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_tl, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_tl, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_tl, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_tl, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_tl, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_tl, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_tl, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_tl, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_tl, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_tl, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_tl, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_tl, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_tl, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_tl, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_tl, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_tl, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_tl, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_tl, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_tl, zero_division= 0))

Logistic Regression Precision: 0.5397923875432526
Decision Tree Precision: 0.2579505300353357
Tuned Decision Tree Precision: 0.344
KNeighborsClassifier Precision: 0.3548387096774194
Tuned KNeighborsClassifier Precision: 0.47368421052631576
GaussianNB Precision: 0.20999128666860295
SVM Precision: 0.15625
Random Forest Precision: 0.43478260869565216
Tuned Random Forest Precision: 0.5797101449275363
AdaBoost Precision: 0.5325077399380805
Tuned AdaBoost Precision: 0.5419354838709678
GradientBoosting Precision: 0.51953125
Tuned GradientBoosting Precision: 0.51953125
XGB Precision: 0.4809384164222874
Tuned XGB Precision: 0.5294117647058824
LGBM Precision: 0.5186721991701245
Tuned LGBM Precision: 0.8571428571428571
CatBoost Precision: 0.46601941747572817
Tuned CatBoost Precision: 0.75


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_tl, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_tl, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_tl, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_tl, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_tl, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_tl, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_tl, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_tl, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_tl, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_tl, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_tl, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_tl, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_tl, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_tl, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_tl, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_tl, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_tl, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_tl, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_tl, zero_division= 0)
                  }

tl_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
tl_precision= tl_precision.sort_values(by= 'Precision Score', ascending=False)
print(tl_precision)

                                    Model  Precision Score
16                  Tuned LGBM Precision:         0.857143
18              Tuned CatBoost Precision:         0.750000
8          Tuned Random Forest Precision:         0.579710
10              Tuned AdaBoost Precision:         0.541935
0          Logistic Regression Precision:         0.539792
9                     AdaBoost Precision:         0.532508
14                   Tuned XGB Precision:         0.529412
11            GradientBoosting Precision:         0.519531
12      Tuned GradientBoosting Precision:         0.519531
15                        LGBM Precision:         0.518672
13                         XGB Precision:         0.480938
4   Tuned KNeighborsClassifier Precision:         0.473684
17                    CatBoost Precision:         0.466019
7                Random Forest Precision:         0.434783
3         KNeighborsClassifier Precision:         0.354839
2          Tuned Decision Tree Precision:         0.3440

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_tl),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_tl),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_tl),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_tl),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_tl),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_tl),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_tl),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_tl),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_tl),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_tl),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_tl),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_tl),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_tl),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_tl),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_tl),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_tl),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_tl),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_tl),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_tl)
                  }

tl_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
tl_accuracy= tl_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(tl_accuracy)

                                   Model  Accuracy Score
10              Tuned AdaBoost Accuracy:        0.906035
0          Logistic Regression Accuracy:        0.905716
9                     AdaBoost Accuracy:        0.905504
8          Tuned Random Forest Accuracy:        0.904444
11            GradientBoosting Accuracy:        0.904338
12      Tuned GradientBoosting Accuracy:        0.904338
15                        LGBM Accuracy:        0.904232
16                  Tuned LGBM Accuracy:        0.903807
18              Tuned CatBoost Accuracy:        0.903701
14                   Tuned XGB Accuracy:        0.903383
4   Tuned KNeighborsClassifier Accuracy:        0.903065
13                         XGB Accuracy:        0.901898
17                    CatBoost Accuracy:        0.901050
6                          SVM Accuracy:        0.900944
7                Random Forest Accuracy:        0.900095
2          Tuned Decision Tree Accuracy:        0.899141
3         KNeighborsClassifier 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_tl),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_tl),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_tl),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_tl),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_tl),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_tl),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_tl),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_tl),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_tl),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_tl),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_tl),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_tl),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_tl),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_tl),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_tl),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_tl),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_tl),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_tl),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_tl)
                  }

tl_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
tl_auc= tl_auc.sort_values(by= 'AUC Score', ascending=False)
print(tl_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.854295
11            GradientBoosting AUC:   0.853766
12      Tuned GradientBoosting AUC:   0.853766
9                     AdaBoost AUC:   0.853364
10              Tuned AdaBoost AUC:   0.851950
15                        LGBM AUC:   0.850389
8          Tuned Random Forest AUC:   0.849522
17                    CatBoost AUC:   0.845560
16                  Tuned LGBM AUC:   0.842714
14                   Tuned XGB AUC:   0.839574
18              Tuned CatBoost AUC:   0.835679
13                         XGB AUC:   0.831847
7                Random Forest AUC:   0.824388
4   Tuned KNeighborsClassifier AUC:   0.817245
5                   GaussianNB AUC:   0.806020
2          Tuned Decision Tree AUC:   0.783918
3         KNeighborsClassifier AUC:   0.711771
6                          SVM AUC:   0.704482
1                Decision Tree AUC:   0.610111
